In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import html5lib

path = 'example.html'
with open(path, 'rt') as myfile:
    data = myfile.read().replace("<br>", '\n')


df = pd.read_html(data)

In [2]:
# Since the older version cape uses a much simpler HTML tree, this is wayy easier to clean
pd.read_html("07capessample.html")[0]

,0,1,2,3,4,5
0,VIS,VIS 60,(A),"Michaelis, Danielle Ruth","Michaelis, Danielle Ruth",SU07
1,NaN,NaN,NaN,Enrollment: 19,Questionnaire's Returned: 18,SelfCAPE


In [3]:
df[3]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,NaN,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,0,0.00,NaN


In [4]:
soup = BeautifulSoup(open("example.html", "r"))

In [5]:
import re

In [6]:
# seperate question, column head, response for each question using dictionaries --> then create dataframe to fit them

In [26]:
def question_collector(bs4obj):
    question_text = bs4obj.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_.*")})
    dct_key = []
    return_dct = {}
    for qt in question_text:
        if qt.text.split()[0].isupper():
            dct_key.append(qt.text)
            return_dct[qt.text] = []
        else:
            return_dct[dct_key[-1]].append(qt.text)
    return return_dct

In [27]:
question_collector(soup)

{'PLEASE COMMENT ON THE FOLLOWING:': ['Your class level is',
  'Your reason for taking this class is',
  'What grade do you expect in this class?'],
 'GENERAL QUESTIONS': [],
 'I learned a great deal from this course.': ['How many hours a week do you spend studying outside of class on average?',
  'How often do you attend this course?'],
 'COURSE MATERIAL AAS 10': ['The course material is intellectually stimulating.',
  'Assignments promote learning.',
  'Required reading is useful.',
  'This course is difficult relative to others.',
  'Exams are representative of the course material.',
  'Do you recommend this course overall?'],
 'INSTRUCTOR Elizabeth Butler': ['Instructor displays a proficient command of the material.',
  'Instructor is well prepared for classes.',
  "Instructor's speech is clear and audible.",
  'Instructor explains the course material well.',
  'Lectures hold your attention.',
  'Instructor’s lecture style facilitates note-taking.',
  "Instructor shows concern for 

In [37]:
def col_collector(bs4obj, lst):
    return None

In [42]:
soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts.*")})

[<span id="ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_1_lblChoiceText_0">Freshman</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_1_lblChoiceText_1">Sophomore</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_1_lblChoiceText_2">Junior</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_1_lblChoiceText_3">Senior</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_1_lblChoiceText_4">Graduate</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_1_lblChoiceText_5">Extension</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_1_lblChoiceText_6">Visitor</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_2_lblChoiceText_0">Major</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_2_lblChoiceText_1">Minor</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_2_lblChoiceText_2">Gen. Ed.</span>,
 <span id="ContentPlaceHolder